In [62]:
import pandas as pd
import numpy as np
from pycocotools.coco import COCO
import os
import cv2
import json
import skimage.io as io
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from wbf_ensemble import *

In [63]:
test_folder = '/data/ephemeral/home/data/medical/img/test/'
get_img_files = os.listdir(test_folder)
if get_img_files[-1].startswith('.'):  # '.'으로 시작하는 파일 제거용
    get_img_files.pop(-1)

In [64]:
len(get_img_files)

100

In [65]:
coco_data = {
        "images": [],
        "annotations": [],
        "categories": [{"id": 1, "name": "text"}],    # 이부분은 비워놔도 됨(only text)
}

for i, test_file in enumerate(get_img_files):
    img = cv2.imread(test_folder+test_file)
    h, w, c = img.shape
    coco_image = {
            "id": i,
            "width": w,
            "height": h,
            "file_name": test_file,
            "license": 0, 
            "flickr_url": None, 
            "coco_url": None, 
            "date_captured": None
    }
    coco_data["images"].append(coco_image)

with open('/data/ephemeral/home/data/medical/output_ensemble/test.json', 'w') as f:
    json.dump(coco_data, f)

In [66]:
submission_files = []
for file_name in os.listdir('/data/ephemeral/home/data/medical/ensemble/json'): # 앙상블 파일
    if file_name.startswith('.'):
        continue
    submission_files.append(file_name)

In [67]:
submission_files         

['36.json', '38.json', '39.json', '33.json']

In [69]:
annotation = '/data/ephemeral/home/data/medical/output_ensemble/test.json'
coco = COCO(annotation)

iou_thr = 0.5            # TODO : iou threshold 설정
skip_box_thr = 0.0001    # TODO : skip iou threshold 설정 0.0001

weight = [1.0, 1.0, 2.0, 1.0]

images_anno = {}
for i, test_file in enumerate(get_img_files):
    boxes_list = []
    scores_list = []
    labels_list = []
    image_info = coco.loadImgs(i)[0]
    
    for j, submission in enumerate(submission_files):
        with open('/data/ephemeral/home/data/medical/ensemble/json/'+submission, 'r') as f:
            data = json.load(f)
            images = data['images']
        words = images[image_info['file_name']]
        box_list = []
        for word_id, word_data in words["words"].items():
            # UFO to PascalVOC format
            [tl, tr, br, bl] = word_data["points"]
            xmin = min(tl[0], tr[0], br[0], bl[0])
            ymin = min(tl[1], tr[1], br[1], bl[1])
            xmax = max(tl[0], tr[0], br[0], bl[0])
            ymax = max(tl[1], tr[1], br[1], bl[1])
            box = [xmin/image_info['width'], ymin/image_info['height'], xmax/image_info['width'], ymax/image_info['height']]
            box_list.append(box)
        boxes_list.append(box_list)
        scores_list.append([weight[j]]*len(words["words"].items()))
        labels_list.append([1]*len(words["words"].items()))

    if len(boxes_list):
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        prediction_words = []
        points_list = []
        for idx, (box, score, label) in enumerate(zip(boxes, scores, labels)):
            # PascalVOC to UFO format
            p_xmin = box[0]*image_info['width']
            p_ymin = box[1]*image_info['height']
            p_xmax = box[2]*image_info['width']
            p_ymax = box[3]*image_info['height']
            p_width = p_xmax - p_xmin
            p_height = p_ymax - p_ymin
            p_tl = [p_xmin, p_ymin]
            p_tr = [p_xmin + p_width, p_ymin]
            p_br = [p_xmin + p_width, p_ymin + p_height]
            p_bl = [p_xmin, p_ymin + p_height]
            points = [p_tl, p_tr, p_br, p_bl]
            points_list.append(points)
        prediction_words = {idx: dict(points=p) for idx, p in enumerate(points_list)}
    images_anno[image_info['file_name']] = dict(words=prediction_words)

images_anno = {'images':images_anno}

# 제출용 : csv 파일로 바로 변경
with open('result.csv', 'w') as f:  
    json.dump(images_anno, f, indent=4)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
